In [ ]:
import SimpleITK as sitk
import nibabel as nib
from supporters import *
import pickle
import os
from pydicom import dcmread

In [ ]:
explore_3D_array(seg)

In [ ]:
def load_scan(path):
    slices = [dcmread(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
    for s in slices:
        s.SliceThickness = slice_thickness
    return slices


def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    image = image.astype(np.int16)
    image[image == -2000] = 0
    for slice_number in range(len(slices)):
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
        image[slice_number] += np.int16(intercept)  
    return np.array(image, dtype=np.int16)


INPUT_FOLDER = '../../data/PatientsDCM/'
patients = os.listdir(INPUT_FOLDER)
patients.sort()

PAT = load_scan(INPUT_FOLDER + patients[14])
PAT = get_pixels_hu(PAT)
PAT.shape

In [ ]:
ban_kinh = 150
a = 150
b = 300

top = a - ban_kinh
left = b - ban_kinh
bottom = a + ban_kinh
right = b + ban_kinh

def euclidian_distance(x, y, a, b):
    dis = ((x - a)**2 + (y - b)**2)**(1/2)
    return dis

crop_imgs = []
def circling():
    for index in range(PAT.shape[0]):
        img = PAT[index]
        for x in range(PAT.shape[1]):
            for y in range(PAT.shape[2]):
                if euclidian_distance(x, y, a, b) > ban_kinh:
                    img[x,y] = -2000
        img = img[int(top) : int(bottom), int(left) : int(right)]
        crop_imgs.append(img)
        
circling()
crop_imgs = np.array(crop_imgs)
crop_imgs.shape

In [ ]:
explore_3D_array(crop_imgs)

In [ ]:
# Save into .nii.gz file
def save(index_PAT):
    index_PAT = '00' + str(index_PAT) if index_PAT < 10 else '0' + str(index_PAT)
    out_path = f'../../data/PatientsDCM/Postprocessing/PAT{index_PAT}.nii.gz'
    converted_array = np.array(crop_imgs, dtype=np.float32)
    converted_array = np.transpose(converted_array, (2, 1, 0))
    affine = np.eye(4)
    nifti_file = nib.Nifti1Image(converted_array, affine)
    nib.save(nifti_file, out_path)
    
save(15)

In [ ]:
# Check
img = sitk.ReadImage('../../data/PatientsDCM/Postprocessing/PAT015.nii.gz', sitk.sitkFloat32)
img = sitk.GetArrayFromImage(img)
print(img.shape)
explore_3D_array(img)

In [ ]:
path_to_seg = '../../data/PatientsDCM/Segmentation_Result/PAT015/3D_res/Segmentation.nii' 

seg = sitk.ReadImage(path_to_seg, sitk.sitkFloat32)
seg = sitk.GetArrayFromImage(seg)
print(seg.shape)

In [ ]:
ban_kinh = 150
a = 150
b = 300

top = a - ban_kinh
left = b - ban_kinh
bottom = a + ban_kinh
right = b + ban_kinh

def euclidian_distance(x, y, a, b):
    dis = ((x - a)**2 + (y - b)**2)**(1/2)
    return dis

crop_seg = []
def circling():
    for index in range(seg.shape[0]):
        img = seg[index]
        for x in range(seg.shape[1]):
            for y in range(seg.shape[2]):
                if euclidian_distance(x, y, a, b) > ban_kinh:
                    img[x,y] = 0
        img = img[int(top) : int(bottom), int(left) : int(right)]
        crop_seg.append(img)
        
circling()
crop_seg = np.array(crop_seg)
crop_seg.shape

In [ ]:
explore_3D_array(crop_seg)

In [ ]:
# Save into .nii.gz file
def save(index_PAT):
    index_PAT = '00' + str(index_PAT) if index_PAT < 10 else '0' + str(index_PAT)
    out_path = f'../../data/PatientsDCM/Postprocessing/PAT{index_PAT}_label.nii.gz'
    converted_array = np.array(crop_seg, dtype=np.float32)
    converted_array = np.transpose(converted_array, (2, 1, 0))
    affine = np.eye(4)
    nifti_file = nib.Nifti1Image(converted_array, affine)
    nib.save(nifti_file, out_path)
    
save(15)